# Tensorflow.org

https://www.tensorflow.org/get_started/get_started

In [3]:
# Import dependencies
import tensorflow as tf

In [6]:
# Create a session
sess = tf.Session()

### linear model

In [4]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [7]:
# tf.Variables are not set until you run init
init = tf.global_variables_initializer()
sess.run(init)

In [9]:
# Test it out
print(sess.run(linear_model, {x:[2,4,6,8]}))

[ 0.30000001  0.90000004  1.5         2.10000014]


In [15]:
# Create a loss function to dertermine close/far the model is from the data
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas) # sum the squares of delta
print(sess.run(loss, {x:[2,4,6,8], y:[0,-1,-2,-3]}))

30.0


Now doing it automatically with tf.train API

In [16]:
# Use gradient descent to optimize the result from our loss function
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [21]:
# Training data (data,labels)
x_train = [2,3,4,5]
y_train = [-1,-2,-3,-4]

# Run train 1000 times aka epochs
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})
    
print(sess.run([W, b, loss], {x:x_train, y:y_train}))

[array([-1.], dtype=float32), array([ 1.], dtype=float32), 0.0]


### MNIST

https://www.tensorflow.org/get_started/mnist/beginners (not a beginner to DL but lets start here)

In [22]:
from tensorflow.examples.tutorials.mnist import input_data

In [23]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [30]:
# Tensor dumped things into a fancy dataset for me
mnist.train.images.shape

(55000, 784)

In [32]:
# And one hot encoded (via one_hot=True)
mnist.test.labels.shape

(10000, 10)

In [33]:
# y = softmax(Wx+b)
# weight has shape [784,10] to multiply with our image vector 784 dimensions and output 10 d vectors for classes
# bias of 10 dimenional vector needed to add to output
x = tf.placeholder(tf.float32, shape=[None, 784])
W = tf.Variable(tf.zeros([784, 10])) 
b = tf.Variable(tf.zeros([10]))

In [35]:
# Define the model => y = softmax(Wx+b)
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [36]:
# Define the loss function
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [37]:
# Setup optimizer for training and minimize our loss function
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [38]:
# not sure what interactive session is
sess = tf.InteractiveSession()

In [39]:
# Initialize global variables and tack on run
tf.global_variables_initializer().run()

In [40]:
for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [42]:
# We didn't use mnist.validation data to validate our model's fit
# Review results of model on test data
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9173
